## Imports

In [18]:
import requests
import time
import json
import pandas as pd
import folium
import os
%store -r google_places_API_Key

## Get all H-E-B stores

In [42]:
Texas_muni = pd.read_csv('Municipalities_in_Texas.csv')
Texas_muni = Texas_muni.dropna(subset='Municipality')
cities = Texas_muni['Municipality'].to_list()

In [43]:
# Google Places API endpoint
endpoint_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"

# Your API key
api_key = google_places_API_Key

# Container for all the results
all_results = []

# Iterate over all cities
for city in cities:
    params = {
        'query': 'H-E-B in ' + city,
        'key': api_key
    }
    
    # Make the API request
    res = requests.get(endpoint_url, params=params).json()
    
    # Append the results to our container
    all_results.extend(res['results'])
    
    # Check if there's a next page token and handle pagination
    while 'next_page_token' in res:
        time.sleep(2)  # delay for a short time before using the next page token
        params['pagetoken'] = res['next_page_token']
        res = requests.get(endpoint_url, params=params).json()
        all_results.extend(res['results'])

# Save the results to a JSON file
with open('heb_locations.json', 'w') as outfile:
    json.dump(all_results, outfile)

In [46]:
# Load JSON data
with open('heb_locations.json', 'r') as file:
    data = json.load(file)

# Convert to DataFrame
df = pd.json_normalize(data)

# Keep only the relevant columns
df = df[['name', 'geometry.location.lat', 'geometry.location.lng', 'formatted_address']]

# Rename the columns for clarity
df.columns = ['Name', 'Latitude', 'Longitude', 'Address']

In [51]:
len(df)

4342

In [50]:
df = df.drop_duplicates()

In [56]:
df2 = df[df['Name'].str.contains('H-E-B')]

In [66]:
df2 = df2[df2['Address'].str.contains('United States')]

In [67]:
len(df2)

855

In [68]:
# Create a base map
m = folium.Map(location=[df2['Latitude'].mean(), df2['Longitude'].mean()], zoom_start=6)

# Add markers for each store
for idx, row in df2.iterrows():
    popup_content = f"<strong>Name:</strong> {row['Name']}<br><strong>Address:</strong> {row['Address']}<br><strong>Latitude:</strong> {row['Latitude']}<br><strong>Longitude:</strong> {row['Longitude']}"
    popup = folium.Popup(popup_content, max_width=300)  # Adjust the max_width as needed
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=popup,
        icon=folium.Icon(icon="cloud"),
    ).add_to(m)
    
m.save('index.html')

# Display the map
m

## Map URL Snagger

In [69]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/H-E-B_in_Texas
